In [ ]:
# My code
import fast_gillespie as fg
import fast_gillespie_viz2 as fgv

In [ ]:
#Define cooperative binding trimer

# Rates in terms of interpretable quantities
mean_num_A_monomers = 100.0
mean_num_B_monomers = 100.0  
interaction_param = 100.0
rate1p = mean_num_A_monomers
rate1m = 1.0
rate2p = mean_num_B_monomers
rate2m = 1.0
rate3p = interaction_param/(2*mean_num_A_monomers*mean_num_B_monomers)
rate3m = 1.0

# Print out rate-determining quantities
print('mean_num_A_monomers = ', mean_num_A_monomers)
print('mean_num_B_monomers = ', mean_num_B_monomers)
print('interaction_param = ', interaction_param)

# Create fields
A = fg.Field(name='A', index_dim=1)
B = fg.Field(name='B', index_dim=1)
a = fg.Field(name='a', index_dim=1)
b = fg.Field(name='b', index_dim=1)
c = fg.Field(name='c', index_dim=1)
d = fg.Field(name='d', index_dim=1)
J = fg.Field(name='J', index_dim=2)
K = fg.Field(name='K', index_dim=2)

# Create/annihilate A monomer
R_1p = fg.MonomerCreationRule(name='R_1p', rate=rate1p, particle=A, sites=[a,b])
R_1m = fg.MonomerAnnihilationRule(name='R_1m', rate=rate1m, particle=A, sites=[a,b])

# Create/annihilate B monomer
R_2p = fg.MonomerCreationRule(name='R_2p', rate=rate2p, particle=B, sites=[c,d])
R_2m = fg.MonomerAnnihilationRule(name='R_2m', rate=rate2m, particle=B, sites=[c,d])

# Create/annihilate J_A:a_B:c
R_3p1 = fg.HeterotypicInteractionCreationRule(name='R_3p1', rate=rate3p, A=A, B=B, a=a, b=c, J=J)
R_3m1 = fg.HeterotypicInteractionAnnihilationRule(name='R_3m1', rate=rate3m, A=A, B=B, a=a, b=c, J=J)    

# Create/annihilate K_B:d_A:b
R_3p2 = fg.HeterotypicInteractionCreationRule(name='R_3p2', rate=rate3p, A=B, B=A, a=d, b=b, J=K)
R_3m2 = fg.HeterotypicInteractionAnnihilationRule(name='R_3m2', rate=rate3m, A=B, B=A, a=d, b=b, J=K) 

# Define fock space and rules
space = fg.FockSpace(fields=[A, B, a, b, c, d, J, K])
rules = fg.RuleSet(rules=[R_1p, R_1m, R_2p, R_2m, R_3p1, R_3m1, R_3p2, R_3m2])

In [ ]:
def summary_stats_func():
    stats_dict = dict(
        A_monomers=len(A.indices - (a.indices | b.indices)),
        B_monomers=len(B.indices - (c.indices | d.indices)),
        interactions=len(J.indices) + len(K.indices)
    )
    return stats_dict

# Define, initialize, and run simulation
sim = fg.Simulation(space=space, rules=rules, custom_stats_func=summary_stats_func)
sim.initialize_state()
sim.run(num_steps=30_000, record_every=100)

# Show simulation stats
fgv.show_sim_stats(sim, x_is_time=True)